#### API 요청 함수 작성

In [32]:
import pandas as pd
import requests
def weather(pageno, pagesize):
    url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
    params ={
        'serviceKey' : "gcVGIzO9n58fC27nANUcTmbfzNXTt93wnEIn/0rLgOiW43nsH213u8dz2JOyvgS1iPxeAi3L69eseRiNQOPMJA==",
        'pageNo' : f'{pageno}',
        'numOfRows': f'{pagesize}',
        'dataType' : 'JSON', 
        'dataCd' : 'ASOS', 
        'dateCd' : 'HR',
        'startDt' : '20200901',
        'startHh' : '00', 
        'endDt' : '20220630', 
        'endHh' : '23',
        'stnIds' : '112' }

    response = requests.get(url, params=params)
    data = response.json().get('response').get('body').get('items').get('item')
    df = pd.DataFrame(data)
    return df
    

### API 요청해서 데이터 끌고오기

In [33]:
dataset = []
for i in range(1,7) :
    data = weather(i,999)
    dataset.append(data)

### 변수의 의미를 파악하고 데이터 셋이 붙이기

In [4]:
data = pd.concat(dataset)
data = data.reset_index(drop = True)
index = pd.read_excel('index.xlsx')
index = index[['변수명','변수의미']]
index_dict = dict()
for i in range(len(index)):
    index_dict[index['변수명'][i]] = index['변수의미'][i]
data.rename(columns = index_dict, inplace = True)
data = data[["시간","현지기압","일조","일사","전운량"]]

### 데이터 정제

In [13]:
### 시간 데이터 변환
data["datetime"] = pd.to_datetime(data["시간"])
data.drop(columns = ["시간"],inplace = True)

### 결측치 처리
data["현지기압"] = data["현지기압"].replace("","0")
data["일조"] = data["일조"].replace("","0")
data["일사"] = data["일사"].replace("","0")
data["전운량"] = data["전운량"].replace("","0")

### 수치형 자료로 변경
data["현지기압"] = data["현지기압"].astype(float)
data["일조"] = data["일조"].astype(float)
data["일사"] = data["일사"].astype(float)
data["전운량"] = data["전운량"].astype(float)

### 데이터 저장
data.to_csv("solar_add_actual_weather.csv",encoding = 'utf-8-sig',index=False)